# 05: Duality and Arrangements

*Authors: Felix Espey*

This notebook serves as supplementary learning material for the course **Geometric Algorithms**.
It showcases and explains implementations of algorithms presented in the corresponding lecture, and elaborates on some practical considerations concerning their use.
Furthermore, it offers interactive visualisations and animations.

## Table of Contents

1. Introduction  
2. Setup
3. Duality  
    3.1. Setup  
    3.2. Visualisation  
    3.3. Algorithm(s)
4. Arragements
5. References

## 1. Introduction

This chapter contains two connected topics, **Duality** and **Arragements**.

**Duality** is an alternative approach to think about points and lines, representing one as the other. This allows to use algorithms that solve problems on lines to also slove problems on points and vice versa. For example, if 3 points are on a line, the dual lines of those points all intersect in a single point. So checking if 3 points are on a line is equivalent to checking if 3 lines intersect in a single point

An **Arangement** $\mathcal{A}$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area inbetween the lines of $L$).

Many problems on a set of points can be solved by first transforming those points into lines and then bulding an Arragement on them.

We will first go over duality and some examples to make the concept and look at Arrangments afterwards. Lastly we will look at some applications of Duality to solve certain problems.

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [11]:
from modules.visualisation import (
    VisualisationTool, PointSetInstance, LineSetInstance, LineSegmentSetInstance, DualityPointsMode, DualityLineMode, DualityLineSegmentMode
    )
from modules.geometry import Point, Line, LineSegment



## 3. Duality

In the intro we went over how **Duality** allows us to represent points and lines and vice versa. In the lecture we went over how this conversion works exactly, but lets repeat it here.

A **point** $p = (x, y)$ becomes the line $p^* := (y = p_x x - p_y)$. $p*$ is then called the dual of $p$.
You can see that the x-coordinate of the point becomes the slope of the line, while the y-coordinate becomes the offset of the line in the y direction. This also lets us define the inverse pretty easily.

A **line** $l = mx + b$ becomes the point $l^* = (m, -b)$. The slope becomes the x-coordinate and the negated y-offset
 becomes the y-coordinate.

A **Line Segment** defined by two points $p_1$ and $p_2$ becomes the two cones between the two dual lines $p^*_1$ and $p^*_2$ of those points. 

The transformation from one object to the other preserves certain properties:
- Self inverse: When Transforming an object $x$ into its dual $x^*$, the dual $x^{**}$ of $x^*$ is the original object $x$ again.
- Vertical distances are preserved: Given a point $p$ and a line $l$ that are a vertical distance $m$ apart, the duals $p^*$ and $l^*$ are also a a vertical distance $m$ apart.
- Relative positions are inversed: Given a point $p$ that is below/on/above a line $l$, the dual line $p^*$ is above/through/below the dual point $l^*$
- Intersections: If two lines $l_1, l_2$ intersect in a point $p$, the dual line $p^*$ of the point will go through the dual points $l^*_1, l^*_2$ of the lines
- Collinear: Given three points $p_1, p_2, p_3$ on a line $l$, the dual lines $p_1^*, p_2^*, p_3^*$ will all intersect in the dual point $l^*$

The concept itself is pretty easy to understand, but since it can be hard to visualise, so we will set up a visual aid.

### 3.1 Visualisation

We will now set up our tool to visualize duality.  The transformation of objects into their respective duals is very simple, we just setup a method each for points, lines and line segments. We then use special drawing modes that not only draw the initial input but also the corresponding dual. To make sure the objects and their duals can be easily distinguished from other objects and duals, we will draw each pair in a different color.

In [12]:
def dual_point_(p : Point) -> Line:
    return Line.line_from_m_b(p.x, -p.y)

def dual_line_(l : Line) -> Point | None:
    m_b = l.get_m_b()
    if type(m_b) is None:
        return None
    return Point(m_b[0], -m_b[1])

def dual_lineSegment_(ls : LineSegment) -> tuple[Line,Line]:
    return [dual_point_(ls.upper), dual_point_(ls.lower)]

pI = PointSetInstance(DualityPointsMode())
pI._default_number_of_random_points = 5
pointToLine = VisualisationTool(400, 400, pI)

lI = LineSetInstance(Point(0,0),Point(400,400), DualityLineMode())
lI._default_number_of_random_points = 10
lineToPoint = VisualisationTool(400, 400, lI)

lsI = LineSegmentSetInstance(DualityLineSegmentMode())
lsI._default_number_of_random_points = 4
lineSegmentToLine = VisualisationTool(400, 400, lsI)

After setting up the visualisation above, we can now actually draw everything. Usually the canvas has the origin in the bottom left, but since you might also want to experiment with negative values for x and y we have offset everything so the origin is in the center of the canvas. To make things easier, the canvy (?) also have x and y axis that get shown once you draw your first point.

In [13]:
pointToLine.display()
lineToPoint.display()
lineSegmentToLine.display()

## 4 Arrangements

An **Arrangement** $\mathcal{A}(L)$ is a subdivision of the plane induced by a set $L$ of lines. It has vertices (intersection of the lines of $L$), edges (segments of the lines of $L$) and faces (area inbetween the lines of $L$).

The outer edges and faces of an Arrangement are unbounded. For edges this happens because the lines that make the edge have some last intersection with the other lines of $L$. The edge after that intersection has no other endpoint. The outer faces are also unbounded as they have no additional line limiting their size.

Since DCELs can only store bounded edges and faces, we generate a bounding box $\mathcal{B}$ around $L$ and only consider the parts of lines inside this box. This adds additional vertices where the lines intersect with the bounding box. We also get new edges as the boundary of the box becomes part of the DCEL.

We start with a DCEL containing only 4 vertices and 4 edges from the bounding box and add the lines of $L$ incrementally. We do this by following along the line, splitting each face we find on the way by adding new Edges and Vertices as needed. 

The steps to computing the Arrangement $\mathcal{A}(L)$ then are as follows:
1. compute the bounding box $\mathcal{B}(L)$
2. initialize DCEL with the edges, vertices and interior face from the bounding box
3. incrementally add the lines of $L$ into the DCEL
   
We start with importing everything we need for this section and setting up the visulation tool, including some example problems.

In [14]:
from modules.geometry import Rectangle, Point, Line, EPSILON, dual_points, dual_lines,Orientation as ORT
from itertools import combinations
from modules.data_structures.animation_objects import BoundingBoxAnimator, DCELAnimator
from modules.visualisation import VisualisationTool, LineSetInstance, BoundingBoxMode, DCELMode, offsetLines, offsetPoints
from modules.data_structures import Vertex, HalfEdge, Face, DoublyConnectedEdgeList as DCEL

visualisation_tool = VisualisationTool(400, 400, LineSetInstance(Point(0,0), Point(400,400)))
bounding_box_mode = BoundingBoxMode()
dcel_mode = DCELMode()

left, right, lower, upper, offset = 50, 350, 50, 350, 50
four_lines = [Line(Point(10,200), Point(200, 390)), Line(Point(200, 390), Point(390,200)), Line(Point(390,200), Point(200, 10)), Line(Point(10,200), Point(200, 10))]
visualisation_tool.register_example_instance("four_lines", four_lines)
diagonales_1 = [Line(Point(48,200), Point(238, 390)), Line(Point(86,200), Point(276, 390)),
                 Line(Point(124,200), Point(314, 390)), Line(Point(162,200), Point(352, 390)), 
                 Line(Point(200,200), Point(390, 390)), Line(Point(200,162), Point(390, 352)), 
                 Line(Point(200,124), Point(390, 314)), Line(Point(200,86), Point(390, 276)),
                 Line(Point(200,48), Point(390, 238))]
diagonales_2 = [Line(Point(200, 390), Point(390,200)), Line(Point(200, 352), Point(390,162)),
                Line(Point(200, 314), Point(390,124)), Line(Point(200, 276), Point(390,86)), 
                Line(Point(200, 238), Point(390,48)), Line(Point(162, 238), Point(352,48)),
                Line(Point(124, 238), Point(314,48)), Line(Point(86, 238), Point(276,48)),
                Line(Point(48, 238), Point(238,48)), Line(Point(10, 238), Point(200,48))]
line_grid = four_lines + diagonales_1 + diagonales_2
visualisation_tool.register_example_instance("grid_example", line_grid)


line_star = four_lines + [Line(Point(10, 10), Point(390,390)), Line(Point(10, 390), Point(390,10)), Line(Point(10, 200), Point(390,200))]
visualisation_tool.register_example_instance("star_example", line_star)



### 4.1 Bounding box

As stated above, the first step is to calculate the bounding box $\mathcal{B}$ of the set of lines $L$.

This bounding box should contain all vertices of the Arrangment $\mathcal{A}(L)$ of $L$. Since the vertices of $\mathcal{A}(L)$ are just the intersection points of the lines in $L$, what we actually need to find are the maximum and minimum x and y coordinates of the intersections. Then we can use them as cornerpoints for an axis alinged box that contains all vertices.

To find the intersection we points we could use a sweepline algorithm, but in this case we will just go with a bruteforce Algorithm that runs in quadratic time. In theory this is super easy, but in pratice we need to be careful with special cases. The steps are as follows:

1. First we need to find all Intersections between the lines. We will use a bruteforce approach for this.
2. Search the list of intersections for the biggest and smallest x and y coordinates.
3. Build initial DCEL from the corners of the bounding box

You can see the code for this below. We loop over all combinations of lines and find their intersection. The intersection of two lines has 3 different possible types: None, Point and Line. For our case only the Point intersections are relevant, so we ignore the other two types. We also add in an additional constraint that the intersections we use must be inside the visible area. This is only so we can guarantee to fully see the bounding box and would not be part of the algorthim in general.

In [15]:


def computeBoundingBox(lines:set[Line], visibleArea : Rectangle) -> BoundingBoxAnimator:
    bbA : BoundingBoxAnimator = BoundingBoxAnimator()
    for line1, line2 in combinations(lines, 2):
        intersection = line1.intersection(line2)
        if(type(intersection) is Point and visibleArea.isInside(intersection)):
            bbA.check_candidate(intersection)
    return bbA

def computeBoundingBoxAlg(lines:set[Line]) -> BoundingBoxAnimator:
    return computeBoundingBox(lines, Rectangle(Point(1,1), Point(399,399)))


visualisation_tool.register_algorithm("Bounding Box", computeBoundingBoxAlg, bounding_box_mode)
visualisation_tool.display()

### 4.2 Arrangement

First we need to be able to find intersections between Lines and Edges, which are effectivly Line Segments. To avoid duplicate code we just transform the Edge into a Line by removing the endpoints. If the intersection is a Point, we then have 3 different possibilites:

1. The intersection is very close to one of the endpoints
2. The intersection is inbetween the endpoints
3. The intersection is outside the endpoints

In the first case we assume the line goes through the respective Vertex and just return that. In the secondcase we return the intersection as a point. Since we return a Vertex in one case and a Point in the other we will later have an easy time distinguishing the cases.

The last case just means the line goes past the Edge on either side, so we do not have any intersection and just return None.

In [16]:
def line_edge_intersection(line : Line, edge : HalfEdge) -> Vertex | Point | None:
    e_as_line = Line(edge.origin.point, edge.destination.point)
    intersection = line.intersection(e_as_line)
    if type(intersection) is Point:
        if(intersection.distance(edge.origin.point) < EPSILON):
            return edge.origin
        elif(intersection.distance(edge.destination.point) < EPSILON):
            return edge.destination
        elif(intersection.orientation(edge.origin.point, edge.destination.point) is ORT.BETWEEN):
            return intersection
    return None

Next up are the methods that actually calculate the Arrangement.

We start with *calc_boundingBox_intersection*. This method takes the DCEL and a Line and returns either the Vertex or the HalfEdge/Point combination where the Line intersects with the DCELs boundingBox. We start by getting the bottom left Vertex from the DCEL and finding an outgoing edge that is adjacent to the outer face. We then follow the next pointers of the edge, which makes sure the adjacent face stays the same, making us follow the outer border of the DCEL. 

Whenever we go to the next edge we check if it has an intersection with the line. If an Intersection is found in an adjacent Vertex, we return that Vertex, if an intersection is found in an Edge we return the Edge and the intersection Point. 

Usually it would not be possible to have a line with no intersections at all since the bounding box is generated from those intersections. But since we discarded some of the intersections to keep the bounding box in the visible are it is possible that a line does not intersect with any Edge in the bounding box. In this case we return None.

In [17]:
def calc_boundingBox_intersection(line : Line, dcel : DCELAnimator) -> Vertex | tuple[HalfEdge, Point] | None:
    outer_face = dcel.get_outer_face()
    v = dcel.get_bottom_left()
    #find first edge of bounding box
    edge = v.edge
    while (not (edge.incident_face is outer_face)):
        edge = edge.twin.next
    #check edges on bounding box for intersection
    while(not (edge.destination is v)):
        intersection = line_edge_intersection(line, edge)
        if (not intersection is None):
            if (type(intersection) is Vertex):
                return intersection
            else:
                return (edge, intersection)
        edge = edge.next
    return None

Once we have found a line-edge intersection we need to follow the line through the arrangement. From the previous step we either have a Vertex or an Edge and a Point on that Edge. If the line intersected and Edge we are good to go, we move to the twin of the edge and have found the face that gets split by the Line. But if the Line happens to go through an already existing Vertex we now need to find which of the faces adjacent to the vertex is one we will need to split.

To find the correct face we will use the two methods below. The *point_between_edge_and_next* method checks if a point is in the cone spanned by an edge and the next edge in the cycle. If the point is in the given cone, this also means that any line from the vertex to that point must also split the face between the edges. In the *find_leaving_edge* method we then use this method to check every ingoing edge of the Vertex. Since the Line goes through the Vertex, we know it splits exactly two Faces around it, one of which we came from and the other being the next face we have to split. Since we know the Face we where at previously we just check if we found a different Face and only return the corresponding Edge in that case.

The choice of Points given to the methods are also important. We need to make sure that the Points are from different sides of the Vertex, otherwise we can miss the correct Face. To make sure we always have Points on different sides of the Vertex we move the two Points that make up the frame outside the window. This way it is guaranteed that one point will be left or above the vertex while the other is right or below.

In [18]:
def point_between_edge_and_next(point: Point, edge: HalfEdge) -> bool:
        edge_0, edge_1 = edge, edge.next
        edge_0_origin, edge_0_destination = edge_0.origin.point, edge_0.destination.point
        edge_1_origin, edge_1_destination = edge_1.origin.point, edge_1.destination.point
        
        if edge_0.twin is edge_1:
            return True
        #point is left of both edges
        caseA1 = point.orientation(edge_0_origin, edge_0_destination) == ORT.LEFT and point.orientation(edge_1_origin, edge_1_destination) == ORT.LEFT# Case A
        #point is left of the first edge and edges make a right turn
        caseA2 = point.orientation(edge_0_origin, edge_0_destination) == ORT.LEFT and edge_1_destination.orientation(edge_0_origin, edge_0_destination) == ORT.RIGHT
        #point is left of second edge and edges make a right turn
        caseB = (point.orientation(edge_1_origin, edge_1_destination) == ORT.LEFT and edge_0_origin.orientation(edge_1_origin, edge_1_destination) == ORT.RIGHT)
        return caseA1 or caseA2 or caseB # Case C (where?)

def find_leaving_edge(v : Vertex, f : Face, line : Line) -> HalfEdge | None:
    e_start = v.edge.twin
    e_cur = e_start
    if(point_between_edge_and_next(line.p1, e_cur) and not (f is e_start.incident_face)):
        return e_cur
    if(point_between_edge_and_next(line.p2, e_cur) and not (f is e_start.incident_face)):
        return e_cur
    e_cur = e_cur.next.twin
    while(not(e_cur is e_start)):
        if(point_between_edge_and_next(line.p1, e_cur) and not (f is e_cur.incident_face)):
            return e_cur
        if(point_between_edge_and_next(line.p2, e_cur) and not (f is e_cur.incident_face)):
            return e_cur
        e_cur = e_cur.next.twin
    return None

The last step to calculate the Arrangment is to actually trace the Line. The method *trace_line* below does exactly that.

First we make sure have and Edge to start at. This is either given as a Parameter if our Line intersected with an Edge, or we have to find it given a Vertex with the method above. Afterwards we use the internal while loop to follow the next pointers of the edge until we find the next edge intersected by the Line. If the intersection is and Edge we add a Vertex and an Edge from the previous Vertex to the new one and the use the twin Pointer of the edge to move to the adjacent Face. If the intersection is a Vertex we just add the Edge and find the next edge with the methods above. We repeat this process till we find an edge of the outer face. 



In [19]:
def trace_line(e_init : HalfEdge, v_init : Vertex, line : Line, dcel : DCELAnimator):
    if e_init is None:
        e_init = find_leaving_edge(v_init, dcel.get_outer_face(), line)
        if e_init is None:
            return
        e_cur = e_init
    else:
        e_cur = e_init.twin
    v_cur = v_init
    while(not e_cur.incident_face is dcel.get_outer_face()):
        intersection = None
        while(intersection is None or e_cur.origin is v_cur or e_cur.destination is v_cur):
            e_cur = e_cur.next
            dcel.animate_edge(e_cur.origin.point, e_cur.destination.point)
            intersection = line_edge_intersection(line, e_cur)
        if type(intersection) is Vertex:
            dcel.add_edge(v_cur.point, intersection.point)
            v_cur = intersection
            e_cur = find_leaving_edge(intersection, e_cur.incident_face, line)
            if e_cur is None:
                return
        elif type(intersection) is Point:
            v_new = dcel.add_vertex_on_edge(intersection, e_cur)
            dcel.add_edge(v_new.point, v_cur.point)
            v_cur = v_new
            e_cur = e_cur.twin.next

def computeDCEL(lines : set[Line]) -> DCELAnimator:
    bbA : BoundingBoxAnimator = computeBoundingBoxAlg(lines)
    dcel : DCELAnimator = DCELAnimator(bbA.boundingBox)
    if dcel.illformed:
        return dcel
    for line in lines:
        intersection = calc_boundingBox_intersection(line, dcel)
        if type(intersection) is Vertex:
            trace_line(None, intersection, line, dcel)
        elif type(intersection) is tuple:
            v = dcel.add_vertex_on_edge(intersection[1], intersection[0])
            trace_line(intersection[0], v, line, dcel)
    return dcel

visualisation_tool.register_algorithm("Arrangement", computeDCEL, dcel_mode)
visualisation_tool.display()


# 5. Examples

Now that we went over Duality and Arrangements in detail, lets look at some examples where one or both can be used to solve problems easier than with a classical approach.

Our first example is an algorithm that takes a set of points and checks if any three points within that set are on a line. First off all, lets remember that since our points use float coordinates, they will never be exactly on a line, so we will have to use an epsilon inequalities. A simple algorithm would then calculate a line between 2 points and see if any of the other points is on that line. This has a worst-case runtime of $O(N^3)$.

But with duality, we can solve the problem much faster. The dual formulation of this problem is: Are there three lines that intersect in the same point. To calculate this we will calculate the Arrangement on the dual lines and stop once the same Vertex is intersected by three lines. 

For inner vertices, every line going through that vertex will generate two edges, so any Vertex with more than four edges is relevant. For outer edges we have two edges from the start, those generated by the bounding box, but every line going through the vertex only adds one additional edge. So we again have to stop once we have five or more edge attached to a vertex. 

The code below is very similar to the Arragement from before, but has added checks to see if newly created edges are lead to the vertex having five or more edges total. When such a vertex is found, the vertex and points corresponding to the intersecting lines are highlighted.

In [20]:
def computeDCEL_3Points(lines : set[Line]) -> DCELAnimator:
    bbA : BoundingBoxAnimator = computeBoundingBoxAlg(lines)
    dcel : DCELAnimator = DCELAnimator(bbA.boundingBox)
    if dcel.illformed:
        return dcel
    for line in lines:
        intersection = calc_boundingBox_intersection(line, dcel)
        if type(intersection) is Vertex:
            #check if 3 or more lines intersect in the vertex
            #we check for >= 4 here because the edge of the new line never gets added
            if len(intersection.outgoing_edges()) >= 4:
                dcel.add_point(intersection.point)
                points = offsetPoints(dual_lines(offsetLines([Line(e.origin.point, e.destination.point) for e in vert.outgoing_edges()], False)), True)
                points = points + offsetPoints(dual_lines(offsetLines([line], False)), True)
                for point in points: 
                    dcel.add_point(point)
                return dcel
            
            vert = trace_3_points_on_line(None, intersection, line, dcel)
            #during line trace, a three-line intersection was found
            if not (vert is None):
                dcel.add_point(vert.point)
                points = offsetPoints(dual_lines(offsetLines([Line(e.origin.point, e.destination.point) for e in vert.outgoing_edges()], False)), True)
                for point in points: 
                    dcel.add_point(point)
                return dcel
        elif type(intersection) is tuple:
            v = dcel.add_vertex_on_edge(intersection[1], intersection[0])
            vert = trace_3_points_on_line(intersection[0], v, line, dcel)
             #during line trace, a three-line intersection was found
            if not (vert is None):
                dcel.add_point(vert.point)
                points = offsetPoints(dual_lines(offsetLines([Line(e.origin.point, e.destination.point) for e in vert.outgoing_edges()], False)), True)
                for point in points: 
                    dcel.add_point(point)
                return dcel
    return dcel

def trace_3_points_on_line(e_init : HalfEdge, v_init : Vertex, line : Line, dcel : DCELAnimator) -> Vertex | None:
    if e_init is None:
        e_init = find_leaving_edge(v_init, dcel.get_outer_face(), line)
        e_cur = e_init
    else:
        e_cur = e_init.twin
    v_cur = v_init
    while(not e_cur.incident_face is dcel.get_outer_face()):
        intersection = None
        while(intersection is None or e_cur.origin is v_cur or e_cur.destination is v_cur):
            e_cur = e_cur.next
            dcel.animate_edge(e_cur.origin.point, e_cur.destination.point)
            intersection = line_edge_intersection(line, e_cur)
        if type(intersection) is Vertex:
            dcel.add_edge(v_cur.point, intersection.point)
             #check if after adding the new edge, the vertex now has 5 or more outgoing edges
            if len(intersection.outgoing_edges()) >= 5:
                return intersection
            v_cur = intersection
            e_cur = find_leaving_edge(intersection, e_cur.incident_face, line)
            if e_cur is None:
                return None
        elif type(intersection) is Point:
            v_new = dcel.add_vertex_on_edge(intersection, e_cur)
            dcel.add_edge(v_new.point, v_cur.point)
            v_cur = v_new
            e_cur = e_cur.twin.next
    return None

def three_points_on_line(points : set[Point]) -> DCELAnimator:
    points = offsetPoints(points, False)
    duals : set[Line] = dual_points(points)
    duals = offsetLines(duals, True)
    return computeDCEL_3Points(duals)
    
example_1 = [Point(180, 50), Point(220, 50), Point(180, 350), Point(220, 350)]
example_2 = example_1 + [Point(180, 220), Point(210, 230), Point(240, 240)]

instance = PointSetInstance(DualityPointsMode())
instance._default_number_of_random_points = 5
visualisation_tool_2 = VisualisationTool(400, 400, instance)
visualisation_tool_2.register_algorithm("3 Points on Line", three_points_on_line, dcel_mode)
visualisation_tool_2.register_example_instance("3_points_not_on_line", example_1)
visualisation_tool_2.register_example_instance("3_points_on_line", example_2)

When trying out the example below, make sure to use the example setups provided as it is very difficult to actually get three points on a line when drawing the in by hand

In [21]:
visualisation_tool_2.display()